# Find info on Acupuncturists in NYC area
## Scrape acufinder.com NYC (Zip code) listings

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

## desired info
- Name
- Degree
- Address (zip in its own column)
- Phone
- Website

In [3]:
url = 'https://www.acufinder.com/list_acupuncturists.php?zip=10011&distance=50&country=62&state=NY&province=&city=&firstname=&lastname=&style=&technique=&expertise=&language=&listtype=paidandfree&association=&searchtype=searchform&srt=rnd&show_all=true&page=1'

In [710]:
def acupunc_me(acufinder):
    url = acufinder
    soup = BeautifulSoup(requests.get(url).content, 'lxml')
    main_table = soup.findAll('td',{'class':'maincopy'})[0]
    aindex = range(1,len(main_table.contents[12].contents)-2,2)
    Name = [main_table.contents[12].contents[i].strong.text for i in aindex]
    Credential = [main_table.contents[12].contents[aindex[i]].text.strip().split(Name[i])[1].split('\n')[0].strip() for i in range(len(aindex))]
    loc_raw = [main_table.contents[12].contents[aindex[i]].contents[1].contents[3].contents[3].contents[1].contents[-9:] for i in range(len(aindex))]
    
    values = [''.join(str(loc_raw[i]).replace("'",'').replace('<br/>','').replace('"','').replace("\\","|").replace('|n','').strip()) for i in range(len(loc_raw))]
    values = [v for v in values if len(v) > 1]
    values = [values[i].split(',')[1:-1] for i in range(len(values))]
    phone = [pd.Series(values).map(lambda x: [i for i in x if len(i) > 3][-1])[j][16:-9].replace('-','') for j in range(len(values))]

    address = pd.Series(values).map(lambda x: ' '.join([i.strip() for i in x if len(i) > 3 and i != "no);return false;>Get Directions</a>"][-5:-3]))
    address = address.map(lambda x: '' if x.find('http://') > 0 else x)

    city = pd.Series(values).map(lambda x: [i.strip() for i in x if len(i) > 3][-5:-2][-1])
    state = pd.Series(values).map(lambda x: [i.strip() for i in x if len(i) > 3][-5:-1][-1].split()[0])#[95][-1].split()[0]
    zip_code = [pd.Series(values).map(lambda x: [i[-5:] for i in x if len(i) > 3][-2])[j] for j in range(len(values))]
    
    
    website = []
    for i in aindex:
        try:
            if main_table.contents[12].contents[i].findAll('a',href=True)[4].text == 'Visit My Website':
                website.append(main_table.contents[12].contents[i].findAll('a',href=True)[4]['href'])
            elif main_table.contents[12].contents[i].findAll('a',href=True)[3].text == 'Visit My Website':
                website.append(main_table.contents[12].contents[i].findAll('a',href=True)[3]['href'])
            else:
                website.append('')
        except:
            website.append('')
            
    acup = pd.DataFrame({'Name':Name,
                        'Credentials':Credential,
                        'Phone':phone,
                        'Address':address,
                        'City':city,
                        'State':state,
                        'Zip Code':zip_code,
                        'Website':website})
    return (acup[['Name','Credentials','Phone','Address','City','State','Zip Code','Website']])

In [728]:
urls = ['https://www.acufinder.com/list_acupuncturists.php?zip={}&distance=50&country=62&state=NY&province=&city=&firstname=&lastname=&style=&technique=&expertise=&language=&listtype=paidandfree&association=&searchtype=searchform&srt=rnd&show_all=true&page=1'.format(nyczip[i]) for i in range(len(nyczip))]

In [730]:
allny = [acupunc_me(urls[i]) for i in range(len(urls))]

In [731]:
nydf = pd.concat(allny)

In [737]:
len(nydf.drop_duplicates())

104

In [713]:
nycacup = acupunc_me(url).sort_values(by=['Zip Code'],ascending=True)

In [725]:
nyczip =(nycacup['Zip Code'].unique()).tolist()

In [742]:
nydf.drop_duplicates().sort_values(by=['Zip Code','Name']).to_csv('NYC Acupuncturist.csv',encoding='utf8',index=False)